In [1]:
import json
import glob
import yaml
from tqdm import tqdm

In [2]:
#done
"""
ref_yaml_list=glob.glob("database/html/1102parse/yaml/*.yaml")
len(ref_yaml_list)

ref_list=[]
for path in tqdm(ref_yaml_list):
    with open(path,"r") as f:
        d=yaml.safe_load(f)
        ref_list.append(d)
"""

'\nref_yaml_list=glob.glob("database/html/1102parse/yaml/*.yaml")\nlen(ref_yaml_list)\n\nref_list=[]\nfor path in tqdm(ref_yaml_list):\n    with open(path,"r") as f:\n        d=yaml.safe_load(f)\n        ref_list.append(d)\n'

In [3]:
#done
"""
sel_ref_list=[]

for ref in ref_list:
    if ref["License"].find("Creative Commons")==-1:
        #print(ref["doi"])
        continue

    if len(ref["main"])==0:
        continue

    sel_ref_list.append(ref)

"""

'\nsel_ref_list=[]\n\nfor ref in ref_list:\n    if ref["License"].find("Creative Commons")==-1:\n        #print(ref["doi"])\n        continue\n\n    if len(ref["main"])==0:\n        continue\n\n    sel_ref_list.append(ref)\n\n'

In [4]:
#with open("database/output/1102ref.json","w") as f:
#    json.dump(sel_ref_list,f,indent=4)

In [5]:

with open("database/output/1102ref.json","r") as f:
    sel_ref_list=json.load(f)

In [6]:
context_path="database/output/context0926.json"
with open(context_path,"r") as f:
    context_list=json.load(f)

In [7]:
with open("database/output/qa_dataset1028.json","r") as f:
    qa_list=json.load(f)

In [8]:
#number of questions
len(qa_list)

41915

In [9]:
# number of literatures used for question
qa_ref_list=[q["ref_id"] for q in qa_list]
qa_ref_list=list(set(qa_ref_list))
len(qa_ref_list)

15337

In [10]:
#number of contexts
len(context_list)

56119

In [ ]:

import re
import itertools
from tqdm import tqdm
import re
def clean_text(text):
    pattern = r'\.(?=[A-Za-z])'
    text= re.sub(pattern, '. ', text)
    #連続したピリオドを削除
    text=re.sub(r'\.{2,}', '.', text)
    return text


def conv_text(text):
    lower_text = text.lower()
    cleaned_text = re.sub(r'[^a-z]', '', lower_text)
    return cleaned_text

original_text_dict={}

for ref_id in tqdm(range(len(sel_ref_list))):
    text_list=sel_ref_list[ref_id]["main"]
    break
    #original_text="\n".join([r["text"] for r in sel_ref_list[ref_id]["main"]])
    original_text=clean_text(original_text)
    original_text=conv_text(original_text)

    original_text_dict[f"{ref_id}"]=(original_text)


100%|██████████| 64666/64666 [01:32<00:00, 700.63it/s]


In [12]:
clean_context_list=[]
for context in tqdm(context_list):
    context=clean_text(context)
    context=conv_text(context)
    clean_context_list.append(context)

100%|██████████| 56119/56119 [00:02<00:00, 23149.95it/s]


In [24]:
from tqdm import tqdm
import Levenshtein as lev
import json
import joblib

context_to_ref_id={}




In [29]:
# original_text_dictの各要素について、30文字の部分文字列をキーとしたインデックスを作成
from collections import defaultdict

substring_indices = defaultdict(list)
for ref_id, t2 in original_text_dict.items():
    for start in range(len(t2) - 29):
        substring_indices[t2[start:start+30]].append(ref_id)

cnt = 0
for idx, (t1, o_t1) in tqdm(enumerate(zip(clean_context_list, context_list))):
    if idx % 1000 == 0:
        print(idx, len(context_to_ref_id))
        if idx % 10000 == 0:
            joblib.dump(context_to_ref_id, "database/processed/1103temp.bin")
    
    t1_start = t1[:30]
    # 30文字の部分文字列に基づいてインデックスを参照
    possible_ref_ids = substring_indices.get(t1_start, [])
    for ref_id in possible_ref_ids:
        if t1_start in original_text_dict[ref_id]:
            context_to_ref_id[o_t1] = ref_id
            break
    cnt += 1


KeyboardInterrupt: 

In [22]:


"""

cnt=0
for t1,o_t1 in tqdm(zip(clean_context_list,context_list)):
    if o_t1 in context_to_ref_id:
        continue

    if cnt%1000==0:
        print(cnt,len(context_to_ref_id))
        joblib.dump(context_to_ref_id,"database/processed/1103temp.bin")
    cnt+=1

    for ref_id,t2 in (original_text_dict.items()):
        if t1[:30] in t2:
        #if t2.find(t1[:30])>0:
            context_to_ref_id[o_t1]=ref_id
            break


    #score= lev.distance(t1,t2[:])
    #score_list.append(score)
"""

1377it [00:00, 3583.80it/s]

1000 1933


2937it [00:03, 481.93it/s] 

3000 2043


4043it [00:07, 289.61it/s]

4000 2287


5078it [00:10, 362.43it/s]

5000 2547


5289it [00:11, 478.93it/s]


KeyboardInterrupt: 

In [23]:
len(context_to_ref_id)

2614

In [14]:
with open("database/processed/1103_context0926to1102ref.json","w") as f:
    json.dump(context_to_ref_id,f,indent=4)

In [ ]:
1